In [1]:
#Importing Dependencies
import pandas as pd
import matplotlib.pyplot as plt
import pybaseball as pyb
from datetime import datetime
from mega import mega_df
from mega import mega_df_cleaned
from mega import mega_concat_df
from functions import team_batting_avg
import re




In [3]:
mega_concat_df.head()

,Gm#,Date,Tm,H/A,Opp,W/L,R,RA,Inn,W-L,...,COL_kk,COL_obp,COL_walks,Opp_at_bats,Opp_ba,Opp_hits,OPP_HR_Column,OPP_kk,Opp_obp,Opp_walks
0,1,Monday Apr 1,COL,@,MIL,L-wo,4,5,10.0,0-1,...,8,0.351351,4,UNKNOWN,0.315789,12,3,11,0.333333,1
1,2,Tuesday Apr 2,COL,@,MIL,W,8,4,NaN,1-1,...,5,0.342857,3,UNKNOWN,0.378378,14,2,11,0.390244,2
2,3,Wednesday Apr 3,COL,@,MIL,W,7,3,NaN,2-1,...,5,0.305556,2,UNKNOWN,0.405405,15,3,8,0.463415,4
3,4,Friday Apr 5,COL,H,SDP,W,5,2,NaN,3-1,...,8,0.235294,1,UNKNOWN,0.000000,0,0,0,0.000000,0
4,5,Saturday Apr 6,COL,H,SDP,W,6,3,NaN,4-1,...,4,0.305556,3,UNKNOWN,0.000000,0,0,0,0.000000,0


### Going Through Linear and Logistic Regression Models